In [1]:
import numpy as np
import math
import copy
import time
from sklearn.neighbors import DistanceMetric
import pandas as pd

# defining cities dataframe with city, latitude and longitude
cities = pd.DataFrame({'city':['Jackson','Gulfport','Southaven','Hattiesburg','Biloxi','Meridian'],
                          'lat':[32.2987573,30.3674198,34.9889818,31.3271189,30.3960318,32.3643098],
                          'lon':[-90.1848103,-89.0928155,-90.0125913,-89.2903392,-88.8853078,-88.703656],})
# converting latitude and longitude values from degree to radians
cities['lat'] = np.radians(cities['lat'])
cities['lon'] = np.radians(cities['lon'])
# defining haversine metric for distance calculation
dist = DistanceMetric.get_metric('haversine')
# converting latitude and longitude from dataframe to array
# multiplying the value with Radius of earth in miles
dist = dist.pairwise(cities[['lat','lon']].to_numpy())*3959
# rounding the value to the nearest integer for better calculation
distances = dist.round()
print(distances)

class Node():
    def __init__(self, size, distances_matrix, distance_sort, order, parent_constraints , extra_constraints = None):
        # Represents the number of cities
        self.size = size
        self.distances_matrix = distances_matrix 
        self.distance_sort = distance_sort
        self.order = order
        self.extra_constraints = extra_constraints
        self.constraints = self.determine_constraints(parent_constraints) 
        self.lowerBound = self.LowerBound()
    # This method determines the constraints of a node based on the constraints of the parent and the extra constraint for this node
    def determine_constraints(self,parent_constraints): 
        constraints = copy.deepcopy(parent_constraints) 
        #print(self.extra_constr)
        if self.extra_constraints == None:
            return constraints
        fr = self.extra_constraints[0]
        to = self.extra_constraints[1] 
        constraints[fr][to] = self.extra_constraints[2] 
        constraints[to][fr] = self.extra_constraints[2] 
        for i in range (2):
            constraints = self.removeEdges(constraints)
            constraints = self.addEdges(constraints) 
        #print(constraints)
        return constraints
    def removeEdges(self,constraints): 
        for i in range(self.size):
            t=0
            for j in range(self.size):
                if (i != j) and (constraints[i][j] == 1): 
                    t += 1
           
            if t >= 2:
                for j in range(self.size):
                    if constraints[i][j] == 2: 
                        constraints[i][j] = 0 
                        constraints[j][i] = 0
            for i in range(self.size):
                for j in range(self.size):
                    t=1
                    prev = i
                    fr = j
                    cycle = False
                    nextOne = self.next_one(prev,fr,constraints) 
                    while (nextOne[0]):
                        t += 1
                        next = nextOne [1] 
                        if next == i:
                            cycle = True
                            break
                        if t > self.size:
                            break
                        prev = fr
                        fr = next
                        nextOne = self.next_one(prev,fr,constraints)
                    if (cycle) and (t < self.size) and (constraints[i][j] == 2):
                        constraints[i][j] = 0
                        constraints[j][i] = 0 
            #print(constraints)
            return constraints
    # This methods checks if all but two edges adjacent to a vertex are excluded. If so, these edges are included
    def addEdges(self,constraints): 
        for i in range(self.size):
            t=0
            for j in range(self.size):
                if constraints[i][j] == 0: 
                    t += 1
            if t == self.size - 2:
                for j in range(self.size):
                    if constraints[i][j] == 2: 
                        constraints[i][j] = 1 
                        constraints[j][i] = 1
     
        return constraints
    # Determines whether there exists an included edge that starts in fr and does not end in prev. 
    # If so, it also returns the endpoint of this edge
    def next_one(self,prev,fr,constraints): 
        for j in range(self.size):
            if (constraints[fr][j] == 1) and (j != prev): 
                return [True,j]
        return[False]
    # Determines if a node represents a full tour by checking whether from every vertex there are exactly 2 included edges and all other edges are excluded.
   # This methods calculates the lower bound which is sum over all vertices of the two smallest allowed edges
    def LowerBound(self): 
        lb = 0
        #print(self.constraints)
        for i in range(self.size): 
            lower = 0
            t=0
            for j in range(self.size):
                if self.constraints[i][j] == 1: 
                    lower += self.distances_matrix[i][j]
                    t += 1   
            tt = 0
        while t < 2:
            shortest = self.distance_sort[i][tt]
            if self.constraints[i][shortest] == 2:
                lower += self.distances_matrix[i][shortest]
                t += 1 
            tt += 1
            if tt >= self.size: 
                lower = math.inf
                break
            lb += lower 
        return lb
    def isTour(self):
        for i in range(self.size):
            num_zero = 0
            num_one = 0
            for j in range(self.size):
                if self.constraints[i][j] == 0: 
                    num_zero += 1
                elif self.constraints[i][j] == 1: 
                    num_one += 1
            if (num_zero != self.size - 2) or (num_one != 2): 
                return False
        return True
    # Checks if a node contains a subtour
    def contains_subtour(self): 
        for i in range(self.size):
            next = self.next_one(i,i,self.constraints) 
            if next [0]:
                prev = i
                fr = next [1]
                t=1
                next = self.next_one(prev, fr, self.constraints)
            while next [0]:
                t += 1
                prev = fr
                fr= next[1]
                if (fr == i) and (t < self.size):
                    return True
                next = self.next_one(prev,fr,self.constraints) 
                if t == self.size:
                    return False
        return False
    # Assumes the node represents a tour and returns the length of this tour
    def tourLength(self): 
        length = 0
        fr = 0
        to = self.next_one(fr,fr,self.constraints)[1]
        for i in range(self.size):
            length += self.distances_matrix[fr][to]
            temp = fr
            fr = to
            to = self.next_one(temp,to,self.constraints)[1]
        return length
    # This method determines the next constraint
    def next_constraint(self):
        #print(self.constraints)
        for i in range(self.size):
            for j in range(self.size):
                if self.constraints[i][j] == 2:
                    return [i,j]
            
    # If a node represents a tour , this method returns a string with the order of the vertices in the tour
    def str (self):
        if self.isTour():
            result = '0'
            fr = 0
            to = None
            for j in range(self.size):
                if self.constraints[fr][j] == 1: 
                    to = j
                    result += '-' + str(j)
                    break
            for i in range(self.size - 1):
                for j in range(self.size):
                    if (self.constraints[to][j] == 1) and (j != fr):
                        result += '-' + str(j) 
                        fr = to
                        to = j
                        break
            return result
        else:
            print('This node is not a tour')
class Travelling_salesman():
    def __init__(self,size,distances_matrix,bestTour = math.inf): 
        self.size = size
        self.distances_matrix = distances_matrix
        self.bestTour = bestTour
        self.bestNode = None
        self.bestNodeTime = 0
        self.num_createdNodes = 0
        self.num_prunedNodes = 0
        self.distance_sort = self.distance_sort() 
        self.order = self.order()
    def findSolution(self):
        root = self.create_root()
        self.num_createdNodes += 1
        T1 = time.perf_counter()
        self.BranchAndBound(root)
        T2 = time.perf_counter()
        print('The shortest tour is:', self.bestNode)
        print('It has a length of', self.bestTour, 'miles')
        print('Shortest path found in',T2 - T1, 'seconds')
        print('Number of nodes created:',self.num_createdNodes) 
        print('Number of nodes pruned:',self.num_prunedNodes)
    # Sorts the edges of the distance matrix per row returns matrix
    def distance_sort(self): 
        result = []
        for i in range(self.size):
            result.append([x for (y, x) in sorted(zip(self.distances_matrix[i], list(range(self.size))))])
        return result
    # returns list of pairs [i,j] in order of increasing costs
    def order(self): 
        edges = []
        lengths = []
        for i in range(self.size):
            for j in range(i + 1, self.size): 
                edges.append([i, j]) 
                lengths.append([i,j])
        result = [z for (l, z) in sorted(zip(lengths, edges))] 
        return result
    def create_root(self):
        no_constraints = []
        for i in range(self.size):
            row_i = []
            for j in range(self.size):
                if (i != j):
                    row_i.append(2)
                else:
                    row_i.append(0)
            no_constraints.append(row_i)
        #print(no_constraints)
        root = Node(self.size,self.distances_matrix,self.distance_sort, self.order,no_constraints)
        return root
    def BranchAndBound(self,node): 
        if node.isTour():
            if node.tourLength() < self.bestTour:
                self.bestTour = node.tourLength()
                self.bestNode = node.str()
                self.bestNodeTime = time.perf_counter()
                print('Best solution found so far: ', self.bestNode,'of length', self.bestTour, 'miles')
        else:
            new_constraint = copy.copy(node.next_constraint())
            if(new_constraint == None):
                return new_constraint
            else:
                new_constraint.append(1)
            #print(new_constraint)
            leftChild = Node(self.size,self.distances_matrix,self.distance_sort, self.order,node.constraints, new_constraint) 
            new_constraint[2] = 0
            rightChild = Node(self.size,self.distances_matrix,self.distance_sort, self.order,node.constraints, new_constraint) 
            self.num_createdNodes += 2
            if (leftChild.contains_subtour()) or (leftChild.lowerBound > 2 * self.bestTour):
                leftChild = None
                self.num_prunedNodes += 1
            if (rightChild.contains_subtour()) or (rightChild.lowerBound > 2 * self.bestTour):
                rightChild = None
                self.num_prunedNodes += 1
            if (leftChild != None) and (rightChild == None):
                self.BranchAndBound(leftChild)
            elif (leftChild == None) and (rightChild != None):
                self.BranchAndBound(rightChild)
            elif (leftChild != None) and (rightChild != None):
                if leftChild.lowerBound <= rightChild.lowerBound: 
                    #print(leftChild.lowerBound)
                    if leftChild.lowerBound < 2 * self.bestTour:
                        self.BranchAndBound(leftChild) 
                    else:
                        leftChild = None
                        self.num_prunedNodes += 1
                    if rightChild.lowerBound < 2 * self.bestTour:
                        self.BranchAndBound(rightChild) 
                    else:
                        rightChild = None 
                        self.num_prunedNodes += 1
                else:
                    if rightChild.lowerBound < 2 * self.bestTour:
                        self.BranchAndBound(rightChild) 
                    else:
                        rightChild = None
                        self.num_prunedNodes += 1
                    if leftChild.lowerBound < 2 * self.bestTour:
                        self.BranchAndBound(leftChild) 
                    else:
                        leftChild = None
                        self.num_prunedNodes += 1

T = Travelling_salesman(6,distances)
T.findSolution()

[[  0. 148. 186.  85. 152.  87.]
 [148.   0. 324.  67.  13. 140.]
 [186. 324.   0. 256. 324. 196.]
 [ 85.  67. 256.   0.  69.  80.]
 [152.  13. 324.  69.   0. 136.]
 [ 87. 140. 196.  80. 136.   0.]]
Best solution found so far:  0-1-2-3-4-5-0 of length 1020.0 miles
Best solution found so far:  0-1-4-2-3-5-0 of length 908.0 miles
Best solution found so far:  0-1-4-3-2-5-0 of length 769.0 miles
Best solution found so far:  0-1-4-3-5-2-0 of length 692.0 miles
Best solution found so far:  0-2-5-1-4-3-0 of length 689.0 miles
Best solution found so far:  0-2-5-4-1-3-0 of length 683.0 miles
The shortest tour is: 0-2-5-4-1-3-0
It has a length of 683.0 miles
Shortest path found in 0.04774261699999993 seconds
Number of nodes created: 185
Number of nodes pruned: 19
